In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import datetime
import pickle
print('Versão do TensorFlow: ', tf.__version__)
import platform

In [ ]:
tf.config.list_physical_devices()

In [ ]:
os = platform.system()
path = './angle_datas.pkl'
log_dir = "../../logs/angle/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_model = "./models/angle/model_saved.h5"
save_model_keras = "./models/angle/model_saved.keras"

if os == 'Darwin':  # 'Darwin' é o nome do kernel do macOS
    path = '../../datas/angle_datas.pkl'
    save_model = "../../models/angle/model_saved.h5"
    save_model_keras = "../../models/angle/model_saved.keras"

In [ ]:


# Carregar os dados do arquivo .pkl
with open(path, 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [71]:
%load_ext tensorboard
log_dir = "../../logs/angle/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Construir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(64, activation='relu'),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(3, activation='softmax')  #  
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(np.array(X_train), np.array(y_train), epochs=1000, batch_size=10000, validation_split=0.2, callbacks=[tensorboard_callback])


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7623 - loss: 2.7107 - val_accuracy: 0.7775 - val_loss: 1.8378
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.7608 - loss: 1.9713 - val_accuracy: 0.7775 - val_loss: 1.1417
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.7608 - loss: 1.2345 - val_accuracy: 0.7746 - val_loss: 0.5149
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.7601 - loss: 0.5685 - val_accuracy: 0.2197 - val_loss: 3.7107
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.2377 - loss: 3.5082 - val_accuracy: 0.4798 - val_loss: 0.7881
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.5058 - loss: 0.7609 - val_accuracy: 0.7746 - val_loss: 0.4482
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.7579 - loss: 0.4850 - val_accuracy: 0.7775 - val_loss: 0.6763
Epoch 8/1000
1/1 ━

In [72]:
# Avaliar o modelo
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f'Model Accuracy: {accuracy * 100:.2f}%')

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9990 - loss: 0.0289
Model Accuracy: 99.77%


In [73]:
# Ações
# 0 Nothing
# 1 Girar para a Esquerda
# 2 Girar para a Direita

# GO TO RIGHT
angle = -33
 
class_names = ['Nothing', 'RoteteLeft', 'RotateRight', ]
predictions = model.predict(np.array([ angle]))
 
score = tf.nn.softmax(predictions[0])
print(  "{}-Must be GO TO RIGHT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)
 



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
2-Must be GO TO RIGHT: RotateRight - with a 57.61 percent confidence.


In [ ]:
%tensorboard

In [74]:
model.save(save_model)
model.save(filepath=save_model_keras)

### From here to you only MACOS to test 

In [ ]:

# !pip install tensorflowjs
from tensorflow.keras.models import load_model
import tensorflow as tf

# Carregar o modelo do arquivo .h5
model = load_model('../../models/angle/model_saved.h5')

# Compile o modelo (você pode precisar ajustar a função de perda, otimizador e métricas conforme necessário)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, '../../models/angle/saved_model/')

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model '../../models/angle/saved_model/' ../../models/tf/angle/web_model